# [DFA Preliminaries](./DFASlide.pdf)

In [12]:
nprocs()==1 && addprocs()

@everywhere logis(x) = 1 ./ (1 .+ exp(-x))
@everywhere dlogis(x) = logis(x) .* (1 .- logis(x))

@everywhere function initParams(x,y,n_hidden,max_iter)
    w1 = randn(n_hidden,size(x,1))
    w2 = randn(n_hidden,n_hidden)
    w3 = randn(size(y,1),n_hidden)  
    B1 = rand(n_hidden,1)
    B2 = rand(n_hidden,1)      
    errors = zeros(max_iter)
    outputs = zeros(size(y,2),max_iter)
    return (w1,w2,w3,errors,outputs,B1,B2)
end

@everywhere function ffward(x,y,w1,w2,w3)
    a1 = w1 * x
    h1 = logis(a1)
    a2 = w2 * h1
    h2 = logis(a2)
    ay = w3 * h2
    yhat = logis(ay)
    err = yhat - y
    return(a1,h1,a2,h2,ay,yhat,err)
end

@everywhere function updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
    w1 .-= δa1 * x'
    w2 .-= δa2 * h1'
    w3 .-= err * h2'
end

@everywhere function bp(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,_,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter   
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (w3' * err) .* dlogis(a2)
        δa1 = (w2' * δa2) .* dlogis(a1)
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function fa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*δa2) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function dfa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*err) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function ifa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa1 = (B1.*err) .* dlogis(a1) 
        δa2 = (w2 * δa1) .* dlogis(a2) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end


.
.


In [14]:
x = [1 1 ; 0 1 ; 1 0 ; 0 0]'
y = [0 1 1 0]
n_hidden = 20; # Number of hidden units
max_iter = 500; #Number of learning steps
trials = 30;

dfa_err=@parallel (hcat) for _=1:trials
    (_,dfa_errors) = dfa(x,y,n_hidden,max_iter)
    dfa_errors
end
(dfa_yhat,_) = dfa(x,y,n_hidden,max_iter)

bp_err = @parallel (hcat) for _=1:trials
    (_,bp_errors) = bp(x,y,n_hidden,max_iter)
    bp_errors
end
(bp_yhat,_) = bp(x,y,n_hidden,max_iter)

ifa_err=@parallel (hcat) for _=1:trials
    (_,ifa_errors) = ifa(x,y,n_hidden,max_iter)
    ifa_errors
end
(ifa_yhat,_) = ifa(x,y,n_hidden,max_iter);

fa_err=@parallel (hcat) for _=1:trials
    (_,fa_errors) = fa(x,y,n_hidden,max_iter)
    fa_errors
end
(fa_yhat,_) = fa(x,y,n_hidden,max_iter);

using Plots
l = @layout [
    a b
    c d
    mp
]
gr()
p1=plot(dfa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="DFA")
p2=plot(ifa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="IFA")
p3=plot(fa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="FA")
p4=plot(bp_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="BP")
p5=plot([dfa_err[:,1] ifa_err[:,1] fa_err[:,1] bp_err[:,1]],label=["DFA" "IFA" "FA" "BP"],title="First Trial Error")
p6=plot([mean(dfa_err,2) mean(ifa_err,2) mean(fa_err,2) mean(bp_err,2)],label=["DFA" "IFA" "BP"],title="Mean Trials Error")
plot(p1,p2,p3,p4,p5,size=(900,900),layout=l)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 DFA 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 39.4926,265.738 40.3073,23.3979 41.1221,284.556 41.9369,23.5401 42.7516,284.919 43.5664,23.9118 44.3812,284.94 45.196,24.5461 46.0107,284.944 46.8255,25.5554 
 47.6403,284.946 48.455,26.1323 49.2698,284.951 50.0846,25.3134 50.8993,284.951 51.7141,24.7748 52.5289,284.951 53.3437,24.4343 54.1584,284.951 54.9732,24.1933 
 55.788,284.951 56.6027,24.011 57.4175,284.951 58.2323,23.8676 59.0471,284.95 59.8618,23.7525 60.6766,284.949 61.4914,23.6595 62.3061,284.948 63.1209,23.585 
 63.9357,284.947 64.7504,23.5264 65.5652,284.944 66.38,23.4814 67.1948,284.939 68.0095,23.448 68.8243,284.93 69.6391,23.4242 70.4538,284.913 71.2686,23.4078 
 72.0834,284.88 72.8981,23.3971 73.7129,284.812 74.5277,23.3904 75.3425,284.668 76.1572,23.3864 76.972,284.35 77.7868,23.3842 78.6015,283.641 79.4163,23.383 
 80.2311,282.133 81.0459,23.3825 81.8606,279.486 82.6754,23.3824 83.4902,276.67 84.3049,23.3827 85.1197,275.359 85.9345,23.3835 86.7492,273.94 87.564,23.3853 
 88.3788,266.884 89.1936,23.3949 90.0083,171.63 90.8231,25.037 91.6379,60.4191 92.4526,33.0525 93.2674,38.0675 94.0822,56.3607 94.897,72.2353 95.7117,149.58 
 96.5265,45.1846 97.3413,281.869 98.156,23.3849 98.9708,282.378 99.7856,23.3889 100.6,283.31 101.415,23.4151 102.23,284.222 103.045,23.5171 103.859,284.333 
 104.674,24.9587 105.489,283.615 106.304,60.5658 107.119,283.642 107.933,260.418 108.748,271.762 109.563,274.611 110.378,275.837 111.192,276.48 112.007,276.876 
 112.822,277.155 113.637,277.376 114.451,277.565 115.266,277.734 116.081,277.89 116.896,278.036 117.711,278.175 118.525,278.306 119.34,278.432 120.155,278.553 
 120.97,278.669 121.784,278.78 122.599,278.887 123.414,278.991 124.229,279.09 125.043,279.187 125.858,279.28 126.673,279.369 127.488,279.456 128.303,279.541 
 129.117,279.622 129.932,279.701 130.747,279.778 131.562,279.853 132.376,279.925 133.191,279.996 134.006,280.064 134.821,280.131 135.635,280.196 136.45,280.259 
 137.265,280.32 138.08,280.38 138.895,280.439 139.709,280.496 140.524,280.551 141.339,280.605 142.154,280.658 142.968,280.71 143.783,280.761 144.598,280.81 
 145.413,280.858 146.227,280.905 147.042,280.952 147.857,280.997 148.672,281.041 149.487,281.084 150.301,281.127 151.116,281.168 151.931,281.209 152.746,281.248 
 153.56,281.287 154.375,281.326 155.19,281.363 156.005,281.4 156.82,281.436 157.634,281.471 158.449,281.506 159.264,281.54 160.079,281.573 160.893,281.606 
 161.708,281.638 162.523,281.67 163.338,281.701 164.152,281.731 164.967,281.761 165.782,281.79 166.597,281.819 167.412,281.848 168.226,281.876 169.041,281.903 
 169.856,281.93 170.671,281.957 171.485,281.983 172.3,282.008 173.115,282.034 173.93,282.058 174.744,282.083 175.559,282.107 176.374,282.13 177.189,282.154 
 178.004,282.177 178.818,282.199 179.633,282.221 180.448,282.243 181.263,282.265 182.077,282.286 182.892,282.307 183.707,282.328 184.522,282.348 185.336,282.368 
 186.151,282.388 186.966,282.407 187.781,282.426 188.596,282.445 189.41,282.464 190.225,282.482 191.04,282.5 191.855,282.518 192.669,282.535 193.484,282.553 
 194.299,282.57 195.114,282.587 195.928,282.603 196.743,282.62 197.558,282.636 198.373,282.652 199.188,282.667 200.002,282.683 200.817,282.698 201.632,282.714 
 202.447,282.728 203.261,282.743 204.076,282.758 204.891,282.772 205.706,282.786 206.52,282.8 207.335,282.814 208.15,282.828 208.965,282.841 209.78,282.855 
 210.594,282.868 211.409,282.881 212.224,282.894 213.039,282.907 213.853,282.919 214.668,282.931 215.483,282.944 216.298,282.956 217.113,282.968 217.927,282.98 
 218.742,282.991 219.557,283.003 220.372,283.014 221.186,283.026 222.001,283.037 222.816,283.048 223.631,283.059 224.445,283.07 225.26,283.08 226.075,283.091 
 226.89,283.101 227.705,

# [MNIST with BP vs DFA](./mnist.ipynb)